In [7]:
import datetime
import paramiko
import os


In [5]:
host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

In [6]:
week_end_date_Q4=[str(datetime.date(2018,11,10)+datetime.timedelta(days=7*i)) for i in range(13)]

week_end_date_Q4_2018=[datetime.datetime.strptime(x,"%Y-%m-%d").date() for x in week_end_date_Q4]
week_end_date_Q4_2018=[str(x) for x in week_end_date_Q4_2018 if x<=datetime.date(2018,12,31)]

week_end_date_Q4_2019=[datetime.datetime.strptime(x,"%Y-%m-%d").date() for x in week_end_date_Q4]
week_end_date_Q4_2019=[str(x) for x in week_end_date_Q4_2019 if x>=datetime.date(2019,1,1)]


remote_folder="/mnt/drv5/jian/Big_Lots/DCM/Analysis_Q4_1_to_1/Q4_transaction_data/"

In [19]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
weekly_2018_list=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
weekly_2018_list=[x for x in weekly_2018_list if ("DailySales" in x) and (".txt" in x)]

weekly_2019_list=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
weekly_2019_list=[x for x in weekly_2019_list if ("DailySales" in x) and (".txt" in x)]

both_year_daily_sales=weekly_2018_list+weekly_2019_list

Q4_daily_sales=[]
for week_end_dt in week_end_date_Q4:
    file = [x for x in both_year_daily_sales if week_end_dt in x]
    if len(file)==1:
        Q4_daily_sales=Q4_daily_sales+file
    else:
        print(week_end_dt,"Error")
        
print(len(Q4_daily_sales))

13


In [22]:
for local_path in Q4_daily_sales:
    base_name=local_path
    base_name=os.path.basename(local_path)
    remote_path=remote_folder+base_name
    
    sftp.put(local_path,remote_path)
    
sftp.close()